In [1]:
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
import pandas as pd
from time import sleep, time
from random import randint
import random
import csv
import re
import requests

In [2]:
headers = {"user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36"}

In [3]:
# we take recipes from here
urls = [
    'https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/',
    'https://www.allrecipes.com/recipes/17188/holidays-and-events/thanksgiving/main-dishes/',
    'https://www.allrecipes.com/recipes/84/healthy-recipes/',
    'https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/',
    'https://www.allrecipes.com/',
    'https://www.allrecipes.com/recipes/17188/holidays-and-events/thanksgiving/main-dishes/'
]

recipes_pages = []

In [4]:
# this function takes the urls for the recipes from page_url
def get_recipe_links(page_url):
    response = requests.get(page_url, headers=headers)
    print(response, response.status_code)
    # check the response
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # take urls
        links = soup.find_all('a', class_= 'comp mntl-card-list-items mntl-universal-card mntl-document-card mntl-card card card--no-image')

        return [link.get('href') for link in links]

    else:
        print(f"Failed to retrieve the page: {page_url}")
        return []

for page_url in urls:
    recipes_pages += get_recipe_links(page_url)
    sleep(random.uniform(0.3, 1))

recipes_pages = list(set(recipes_pages))
print(f"Total recipe links found: {len(recipes_pages)}")


<Response [200]> 200
<Response [200]> 200
<Response [200]> 200
<Response [200]> 200
<Response [200]> 200
<Response [200]> 200
Total recipe links found: 254


In [5]:
def extract_int_with_commas(text):
    # find comma seperated int and parse it, return 0
    # if none found
    m = re.match(r'\d{1,3}(,\d{3})*', text)
    if m:
        return int(m.group().replace(',', ''))
    else:
        return 0

# Function to scrape details from a single recipe page
def get_recipe_details(recipe_url):
    response = requests.get(recipe_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        title = soup.find('h1', {'class': 'article-heading text-headline-400'}).text

        # check if was rated
        rating_div = soup.find('div', {'id': 'mm-recipes-review-bar__rating_1-0'})
        if rating_div is not None:
            rating = float(rating_div.text)

            ratings_count_txt = soup.find('div', {'id': 'mm-recipes-review-bar__rating-count_1-0'}).text
            ratings_count = int(re.findall(r'\d+', ratings_count_txt)[0])
        else:
            rating = None
            ratings_count = 0


        reviews_count_txt = soup.find('div', {'id': 'mm-recipes-review-bar__comment-count_1-0'}).text
        # print(reviews_count_txt)
        reviews_count = extract_int_with_commas(reviews_count_txt)

        photos_count_txt = soup.find('div', {'id': 'recipe-review-bar__photo-count_1-0'}).text
        # print(photos_count_txt)
        photos_count = extract_int_with_commas(photos_count_txt)

        ingredients_cnt = len(soup.find_all('li', {'class': 'mm-recipes-structured-ingredients__list-item'}))

        return {
                'url': recipe_url,
                'title': title,
                'rating': rating,
                'num_ratings': ratings_count,
                'num_reviews': reviews_count,
                'num_photos': photos_count,
                'num_ingredients': ingredients_cnt,
        }
    else:
        print(f"Failed to retrieve {recipe_url}")
        return None


In [6]:
recipes_data = []


num_recipes = 100
# Iterate over the recipe URLs and scrape the data
for page_url in recipes_pages:
    if not '/recipe/' in page_url:
        continue

    recipe_details = get_recipe_details(page_url)
    if recipe_details is not None:
        recipes_data.append(recipe_details)
    else:
        continue

    # stop after 100 recipes scraped
    if len(recipes_data) >= num_recipes:
      break

In [7]:
# save the data
df = pd.DataFrame.from_records(recipes_data)
df.to_csv('results.csv')